In [3]:
import arcpy
import numpy as np
from netCDF4 import Dataset

# Define input raster and output NetCDF file
input_raster = r"C:\Users\abiga\OneDrive\Documents\Thesis\GEOG4057_Project\GEOG4057_Project.gdb\PR_Raster"  # Raster created from FeatureToRaster
output_netcdf = r"C:\Users\abiga\OneDrive\Documents\Thesis\PR_boundary.nc"

# Load raster
raster = arcpy.Raster(input_raster)
array = arcpy.RasterToNumPyArray(raster, nodata_to_value=0)  # Convert NoData to 0

# Convert all non-zero values to 1 (binary mask)
binary_array = np.where(array > 0, 1, 0)

# Get raster properties
desc = arcpy.Describe(raster)
extent = desc.extent
spatial_ref = desc.spatialReference

# Approximate bounding box based on Albers extent in meters
# NAD83 Albers projection for contiguous USA: approximate geographic extent
approx_lat_min, approx_lat_max = 18.5171840, 17.8857690
approx_lon_min, approx_lon_max = -67.9429677, -65.2207055

# Scale latitude and longitude dynamically based on raster extent
rows, cols = binary_array.shape

# Use the spatial extent of the raster to determine scaling factor
lat_values = np.linspace(approx_lat_max, approx_lat_min, rows)
lon_values = np.linspace(approx_lon_min, approx_lon_max, cols)

# Save to NetCDF
with Dataset(output_netcdf, "w", format="NETCDF4") as ncfile:
    # Create dimensions
    ncfile.createDimension("lat", rows)
    ncfile.createDimension("lon", cols)
    
    # Create latitude and longitude variables
    lat_var = ncfile.createVariable("lat", "f4", ("lat",))
    lon_var = ncfile.createVariable("lon", "f4", ("lon",))
    
    # Assign coordinate values (1D arrays)
    lat_var[:] = lat_values
    lon_var[:] = lon_values

    # Define raster variable
    mask_var = ncfile.createVariable("mask", "i1", ("lat", "lon"))
    mask_var.units = "binary"
    mask_var.long_name = "Binary mask (1=shape present, 0=absent)"
    
    # Assign binary mask data
    mask_var[:, :] = binary_array

print("NetCDF file saved successfully with corrected lat/lon scale at:", output_netcdf)

NetCDF file saved successfully with corrected lat/lon scale at: C:\Users\abiga\OneDrive\Documents\Puerto_Rico\PR_boundary.nc
